In [1]:
import pandas as pd
import os
import numpy as np
import torch
from torch import nn
from torch import optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [2]:
cuda0 = torch.device("cuda:0")

Загружаем датасет:

In [3]:
!echo 'here you can put your kaggle.json' > kaggle.json
!mkdir /root/.kaggle
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d movsisyanm/armenian-news-articles-tertam

 94% 204M/217M [00:01<00:00, 124MB/s]
100% 217M/217M [00:01<00:00, 123MB/s]


In [5]:
!unzip armenian-news-articles-tertam.zip

Archive:  armenian-news-articles-tertam.zip
  inflating: business.csv            
  inflating: categories.md           
  inflating: corona.csv              
  inflating: culture.csv             
  inflating: events.csv              
  inflating: law.csv                 
  inflating: other.csv               
  inflating: politics.csv            
  inflating: press_digest.csv        
  inflating: sport.csv               


Здесь - 9 файлов с новостями и отдельный файл с названиями категорий.
\
Посмотрим, сколько заголовков в каждом файле:

In [6]:
data = []
len_lst = []
for fname in os.listdir('/content'):
  if fname[-3:] == 'csv':
    data.append(pd.read_csv(fname))
    len_lst.append([fname[:-4], len(data[-1])])

In [7]:
for cat, count in len_lst:
  print("%s : %d" % (cat , count))

politics : 21315
other : 11818
law : 6225
sport : 84346
press_digest : 33757
culture : 21860
business : 31337
corona : 8850
events : 194924


In [8]:
len_lst = list(map(lambda el: el[1], len_lst))

In [9]:
print("min: %d" % min(len_lst))
print("max: %d" % max(len_lst))

min: 6225
max: 194924


In [10]:
np.mean(len_lst)

46048.0

In [11]:
np.std(len_lst)

57092.8273837149

In [12]:
data = pd.concat(data)

In [13]:
data.head()

,title,date,content,category
0,Առողջապահության նախարարի՝ հոկտեմբերի 1-ից ուժի...,2021-09-24 19:14:00,«Հայաստան» խմբակցությունը հաշվի առնելով հանրայ...,4
1,Ադրբեջանը շարունակում է թաքցնել գերիների իրակա...,2021-09-24 20:21:00,Սեպտեմբերի 24-ին Նյու Յորքում ՄԱԿ-ի Գլխավոր աս...,4
2,"Այն, որ ՀՀ-ն հիմա ՀԱՊԿ նախագահող է, մեզ լրացու...",2021-09-23 18:42:00,«Ազատություն» ՌԿ եթերում ՀՀ փոխվարչապետ Սուրե...,4
3,Փաշինյան-Էրդողան հանդիպում նախատեսված չէ. Սուր...,2021-09-23 19:09:00,"Մեկնաբանություն չունեմ, ես հիմա պիտի Էրդողանին...",4
4,Ֆրենկ Փալոնը շնորհավորել է Անկախության տոնի կա...,2021-09-21 22:27:00,ԱՄՆ Կոնգրեսի հայկական հարցերի հանձնախմբի համան...,4


In [14]:
print(data.dtypes)

title       object
date        object
content     object
category     int64
dtype: object


In [15]:
len(data)

414432

В датасете хранятся данные о заголовке, времени, тексте новости и её категории.

Проверим, все ли столбцы заполнены:

In [16]:
data.columns[data.isna().any()].tolist()

['content']

In [17]:
data['content'].isna().sum() # количество пропусков

1107

In [18]:
data.dropna(inplace = True) # удалили строки с пропущенным текстом

In [19]:
data.describe()

,category
count,413325.000000
mean,3.250781
std,2.136672
min,0.000000
25%,2.000000
50%,2.000000
75%,6.000000
max,8.000000


In [20]:
data

,title,date,content,category
0,Առողջապահության նախարարի՝ հոկտեմբերի 1-ից ուժի...,2021-09-24 19:14:00,«Հայաստան» խմբակցությունը հաշվի առնելով հանրայ...,4
1,Ադրբեջանը շարունակում է թաքցնել գերիների իրակա...,2021-09-24 20:21:00,Սեպտեմբերի 24-ին Նյու Յորքում ՄԱԿ-ի Գլխավոր աս...,4
2,"Այն, որ ՀՀ-ն հիմա ՀԱՊԿ նախագահող է, մեզ լրացու...",2021-09-23 18:42:00,«Ազատություն» ՌԿ եթերում ՀՀ փոխվարչապետ Սուրե...,4
3,Փաշինյան-Էրդողան հանդիպում նախատեսված չէ. Սուր...,2021-09-23 19:09:00,"Մեկնաբանություն չունեմ, ես հիմա պիտի Էրդողանին...",4
4,Ֆրենկ Փալոնը շնորհավորել է Անկախության տոնի կա...,2021-09-21 22:27:00,ԱՄՆ Կոնգրեսի հայկական հարցերի հանձնախմբի համան...,4
...,...,...,...,...
194919,Պակիստանում զոհերի թիվն աճում է,2008-09-21 14:57:00,Պակիստանի մայրաքաղաք Իսլամաբադում տեղի ունեցած...,2
194920,Չինաստանում հրդեհի արդյունքում զոհվել են տասնյ...,2008-09-21 15:17:00,Չինաստանի հարավում գտնվող Գուանդոն նահանգի Շեն...,2
194921,Իրաքում ահաբեկչական գործողությունների հետևանքո...,2008-09-22 10:04:00,Իրաքի հյուսիսում իրականացված երկու ահաբեկչությ...,2
194922,Իսպանիան հայտնվել է ահաբեկչությունների կիզակետում,2008-09-22 10:14:00,Իսպանիայի հյուսիսում գտնվող Օնդառոա քաղաքում գ...,2


In [21]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_train):
  for text in data_train:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(data['content']), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])  # какой индекс нужно вернуть, если слово не найдено в словаре

In [22]:
'''
сlass ArmenianNewsDataset(Dataset):
    def __init__(self, data_title, data_date, data_text, y):
        self.title = data_title
        self.date = data_date
        self.text = data_text
        self.y = y

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        label = self.y.iloc[idx]
        out_title = vocab(tokenizer(self.title.iloc[idx]))
        out_date = 0
        out_text = vocab(tokenizer(self.text.iloc[idx]))
        return out_title, out_date, out_text, label
'''

'\nсlass ArmenianNewsDataset(Dataset):\n    def __init__(self, data_title, data_date, data_text, y):\n        self.title = data_title\n        self.date = data_date\n        self.text = data_text\n        self.y = y\n\n    def __len__(self):\n        return len(self.title)\n\n    def __getitem__(self, idx):\n        label = self.y.iloc[idx]\n        out_title = vocab(tokenizer(self.title.iloc[idx]))\n        out_date = 0\n        out_text = vocab(tokenizer(self.text.iloc[idx]))\n        return out_title, out_date, out_text, label\n'

In [23]:
class ArmenianNewsDataset(Dataset):
    def __init__(self, data_text, y):
        self.text = data_text
        self.y = y

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        return vocab(tokenizer(self.text.iloc[idx])), self.y.iloc[idx]

In [24]:
data_train, data_test, y_train, y_test = train_test_split(data['content'], data['category'], test_size=0.3)
trainset = ArmenianNewsDataset(data_text=data_train, y=y_train)
testset = ArmenianNewsDataset(data_text=data_test, y=y_test)

In [43]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(_label)
         processed_text = torch.tensor(_text, dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64, device = cuda0)
    offsets = torch.tensor(offsets[:-1], device = cuda0).cumsum(dim=0)
    text_list = torch.cat(text_list)
    text_list = text_list.to(cuda0)
    return label_list, text_list, offsets

In [44]:
trainloader = DataLoader(trainset, batch_size=8, shuffle=False, collate_fn=collate_batch)
testloader = DataLoader(testset, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [45]:
num_class = len(set(y_train))
vocab_size = len(vocab)
print("Кол-во классов: ", num_class)
print("Размер словаря: ", vocab_size)

Кол-во классов:  9
Размер словаря:  1194648


In [46]:
class TextClassificationModel(nn.Module):
    def __init__(self):
        super(TextClassificationModel, self).__init__()
        self.bag = nn.EmbeddingBag(vocab_size, 64, sparse=True)  # слой-матрица размером 81125 x 64, 81125 - такое количество слов в слловаре
        self.lin = nn.Linear(64, num_class)  # на вход подаётся 64 числа - выход от self.bag, слой выдаёт 4 числа - вероятность каждой новости, их 4 типа

    def forward(self, text, offsets):
        return self.lin(self.bag(text, offsets))

In [87]:
net = TextClassificationModel()
net.to(cuda0)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr= 1)

In [88]:
losses = []
running_corrects = 0
net.train(True)
for epoch in range(3):
    running_loss = 0.0
    running_corrects = 0.0
    for i, data in enumerate(trainloader, 0):
        labels, inputs, offsets = data
        outputs = net(inputs, offsets)
        _, preds = torch.max(outputs.data, 1)
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_corrects += int(torch.sum(preds == labels.data)) / len(labels)
        if i % 10000 == 9999:
          print('[%d, %5d] loss: %.3f accuracy: %.3f' % (epoch + 1, i + 1, running_loss/10000, running_corrects/10000 ))
          losses += [running_loss/10000]
          running_loss = 0.0
          running_corrects = 0.0
print('Finished Training')

[1, 10000] loss: 0.917 accuracy: 0.703
[1, 20000] loss: 0.683 accuracy: 0.762
[1, 30000] loss: 0.627 accuracy: 0.778
[2, 10000] loss: 0.572 accuracy: 0.794
[2, 20000] loss: 0.550 accuracy: 0.799
[2, 30000] loss: 0.539 accuracy: 0.803
[3, 10000] loss: 0.515 accuracy: 0.809
[3, 20000] loss: 0.504 accuracy: 0.812
[3, 30000] loss: 0.499 accuracy: 0.815
Finished Training


In [89]:
net.train(False)
runninig_correct = 0
num_of_tests = 0
for data in testloader:
    labels, inputs, offsets = data
    output = net(inputs, offsets)
    _, predicted = torch.max(output, 1)
    runninig_correct += int(torch.sum(predicted == labels)) / len(labels)
    num_of_tests += 1
print(runninig_correct / num_of_tests)

0.8058118279569892
